# Évaluation des Modèles de Détection de Fraudes

Ce notebook compare et évalue les performances des différents modèles avec des analyses approfondies.

## Objectifs

- Charger les modèles entraînés et les comparer
- Analyser les performances détaillées par métriques
- Étudier les erreurs et les cas limites
- Évaluer la robustesse des modèles
- Générer des rapports de performance complets
- Identifier les axes d'amélioration

In [ ]:
# Configuration et imports
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Ajout du chemin racine au sys.path
ROOT_DIR = Path.cwd().parent
sys.path.append(str(ROOT_DIR))

# Imports de base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import joblib

# Imports ML et métriques
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_curve, auc, precision_recall_curve,
    average_precision_score, brier_score_loss
)
from sklearn.calibration import calibration_curve
from sklearn.model_selection import learning_curve

# Imports locaux
from src.utils.metrics import calculate_metrics, pr_auc_score

# Configuration des graphiques
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Environnement configuré avec succès")

## 1. Chargement des Modèles et Données

In [ ]:
# Chargement du modèle entraîné
print("🔄 Chargement du modèle et des données...")

# Chargement du modèle
model_path = "../models/trained/best_model.pkl"
if os.path.exists(model_path):
    best_model = joblib.load(model_path)
    print("✅ Modèle chargé avec succès")
else:
    print("❌ Modèle non trouvé. Veuillez exécuter le notebook d'entraînement d'abord.")
    raise FileNotFoundError("Modèle non trouvé")

# Chargement des métadonnées
metadata_path = "../models/metadata/model_metadata.json"
if os.path.exists(metadata_path):
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    print("✅ Métadonnées chargées")
else:
    print("⚠️ Métadonnées non trouvées")
    metadata = {}

# Chargement des données de test
features_path = "../data/processed/features_engineered.csv"
if os.path.exists(features_path):
    df = pd.read_csv(features_path)
    print("✅ Données de test chargées")
else:
    print("❌ Données de test non trouvées")
    raise FileNotFoundError("Données de test non trouvées")

print(f"📊 Données : {df.shape}")
print(f"🏆 Modèle chargé : {metadata.get('model_name', 'Unknown')}")

In [ ]:
# Préparation des données de test
print("🔄 Préparation des données de test...")

# Séparation features/cible
X = df.drop('Class', axis=1)
y = df['Class']

# Création d'un jeu de test représentatif
from sklearn.model_selection import train_test_split
_, X_test, _, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 Test set : {X_test.shape}")
print(f"📈 Distribution classes test : {y_test.value_counts(normalize=True).to_dict()}")

# Prédictions du modèle
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

print("✅ Prédictions générées")

## 2. Analyse des Métriques Principales

In [ ]:
# Calcul des métriques détaillées
print("📊 Analyse des métriques principales...")

# Métriques de base
metrics = calculate_metrics(y_test, y_pred, y_proba.reshape(-1, 1))

# Métriques supplémentaires
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score

additional_metrics = {
    'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
    'cohen_kappa': cohen_kappa_score(y_test, y_pred),
    'brier_score': brier_score_loss(y_test, y_proba)
}

# Combinaison des métriques
all_metrics = {**metrics, **additional_metrics}

print("📈 MÉTRIQUES D'ÉVALUATION COMPLÈTES :")
print("-" * 50)
for metric_name, value in all_metrics.items():
    if isinstance(value, float):
        print("25s")
    else:
        print("25s")

print("\n📋 RAPPORT DE CLASSIFICATION DÉTAILLÉ :")
print(classification_report(y_test, y_pred, target_names=['Légitime', 'Frauduleuse']))

In [ ]:
# Visualisation des métriques principales
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Métriques d\'Évaluation - Analyse Détaillée', fontsize=16)

# Métriques à visualiser
viz_metrics = ['precision', 'recall', 'f1', 'roc_auc', 'pr_auc', 'balanced_accuracy']
viz_values = [all_metrics[m] for m in viz_metrics]

# Bar plot des métriques
bars = axes[0, 0].bar(viz_metrics, viz_values, color='skyblue', alpha=0.8)
axes[0, 0].set_title('Métriques Principales')
axes[0, 0].set_ylabel('Score')
axes[0, 0].tick_params(axis='x', rotation=45)

# Ajout des valeurs sur les barres
for bar, value in zip(bars, viz_values):
    axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                    '.3f', ha='center', va='bottom')

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 1],
            xticklabels=['Légitime', 'Frauduleuse'], 
            yticklabels=['Légitime', 'Frauduleuse'])
axes[0, 1].set_title('Matrice de Confusion')
axes[0, 1].set_xlabel('Prédiction')
axes[0, 1].set_ylabel('Réalité')

# Distribution des probabilités
axes[0, 2].hist(y_proba[y_test == 0], alpha=0.7, label='Légitime', bins=50, color='green')
axes[0, 2].hist(y_proba[y_test == 1], alpha=0.7, label='Frauduleuse', bins=50, color='red')
axes[0, 2].set_title('Distribution des Probabilités')
axes[0, 2].set_xlabel('Probabilité de Fraude')
axes[0, 2].set_ylabel('Fréquence')
axes[0, 2].legend()

# Courbe ROC
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)
axes[1, 0].plot(fpr, tpr, color='darkorange', lw=2, label='.2f')
axes[1, 0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[1, 0].set_xlim([0.0, 1.0])
axes[1, 0].set_ylim([0.0, 1.05])
axes[1, 0].set_xlabel('Taux de Faux Positifs')
axes[1, 0].set_ylabel('Taux de Vrais Positifs')
axes[1, 0].set_title('Courbe ROC')
axes[1, 0].legend(loc="lower right")
axes[1, 0].grid(True)

# Courbe Precision-Recall
precision, recall, _ = precision_recall_curve(y_test, y_proba)
pr_auc = auc(recall, precision)
axes[1, 1].plot(recall, precision, color='blue', lw=2, label='.2f')
axes[1, 1].set_xlim([0.0, 1.0])
axes[1, 1].set_ylim([0.0, 1.05])
axes[1, 1].set_xlabel('Rappel')
axes[1, 1].set_ylabel('Précision')
axes[1, 1].set_title('Courbe Precision-Recall')
axes[1, 1].legend(loc="lower left")
axes[1, 1].grid(True)

# Courbe de calibration
prob_true, prob_pred = calibration_curve(y_test, y_proba, n_bins=10)
axes[1, 2].plot(prob_pred, prob_true, marker='o', color='red', label='Modèle')
axes[1, 2].plot([0, 1], [0, 1], linestyle='--', color='gray', label='Calibration parfaite')
axes[1, 2].set_xlabel('Probabilité Prédite')
axes[1, 2].set_ylabel('Probabilité Observée')
axes[1, 2].set_title('Courbe de Calibration')
axes[1, 2].legend()
axes[1, 2].grid(True)

plt.tight_layout()
plt.show()

## 3. Analyse des Erreurs

In [ ]:
# Analyse des erreurs de classification
print("🔍 Analyse des erreurs de classification...")

# Création d'un DataFrame avec les prédictions et erreurs
results_df = X_test.copy()
results_df['y_true'] = y_test
results_df['y_pred'] = y_pred
results_df['y_proba'] = y_proba
results_df['error'] = (y_test != y_pred).astype(int)

# Statistiques des erreurs
print("📊 STATISTIQUES DES ERREURS :")
print(f"Total d'erreurs : {results_df['error'].sum()}")
print(".2%")
print(f"Taux d'erreur classe 0 (Légitime) : {(results_df[(results_df['y_true'] == 0) & (results_df['error'] == 1)].shape[0] / (y_test == 0).sum()) * 100:.2f}%")
print(f"Taux d'erreur classe 1 (Frauduleuse) : {(results_df[(results_df['y_true'] == 1) & (results_df['error'] == 1)].shape[0] / (y_test == 1).sum()) * 100:.2f}%")

# Analyse des faux positifs et faux négatifs
false_positives = results_df[(results_df['y_true'] == 0) & (results_df['y_pred'] == 1)]
false_negatives = results_df[(results_df['y_true'] == 1) & (results_df['y_pred'] == 0)]

print(f"\nFaux positifs : {len(false_positives)}")
print(f"Faux négatifs : {len(false_negatives)}")

In [ ]:
# Analyse des seuils de décision
print("📊 Analyse des seuils de décision...")

thresholds = np.arange(0.1, 0.9, 0.05)
threshold_metrics = []

for threshold in thresholds:
    y_pred_thresh = (y_proba >= threshold).astype(int)
    
    # Calcul des métriques pour ce seuil
    thresh_metrics = calculate_metrics(y_test, y_pred_thresh, y_proba.reshape(-1, 1))
    thresh_metrics['threshold'] = threshold
    threshold_metrics.append(thresh_metrics)

# Conversion en DataFrame
threshold_df = pd.DataFrame(threshold_metrics)

# Visualisation des métriques selon le seuil
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Analyse des Seuils de Décision', fontsize=16)

axes[0, 0].plot(threshold_df['threshold'], threshold_df['precision'], 'b-', label='Precision', linewidth=2)
axes[0, 0].plot(threshold_df['threshold'], threshold_df['recall'], 'r-', label='Recall', linewidth=2)
axes[0, 0].set_title('Precision vs Recall')
axes[0, 0].set_xlabel('Seuil')
axes[0, 0].set_ylabel('Score')
axes[0, 0].legend()
axes[0, 0].grid(True)

axes[0, 1].plot(threshold_df['threshold'], threshold_df['f1'], 'g-', linewidth=2)
axes[0, 1].set_title('F1-Score vs Seuil')
axes[0, 1].set_xlabel('Seuil')
axes[0, 1].set_ylabel('F1-Score')
axes[0, 1].grid(True)

axes[1, 0].plot(threshold_df['threshold'], threshold_df['precision'] * threshold_df['recall'], 'purple', linewidth=2)
axes[1, 0].set_title('Precision × Recall')
axes[1, 0].set_xlabel('Seuil')
axes[1, 0].set_ylabel('Precision × Recall')
axes[1, 0].grid(True)

axes[1, 1].scatter(threshold_df['recall'], threshold_df['precision'], 
                   c=threshold_df['threshold'], cmap='viridis', s=50)
axes[1, 1].set_xlabel('Recall')
axes[1, 1].set_ylabel('Precision')
axes[1, 1].set_title('Courbe Precision-Recall')
axes[1, 1].grid(True)

plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1], label='Seuil')
plt.tight_layout()
plt.show()

# Recommandation de seuil optimal
optimal_idx = threshold_df['f1'].idxmax()
optimal_threshold = threshold_df.loc[optimal_idx, 'threshold']
print(".3f")
print(".4f")

## 4. Analyse de Robustesse

In [ ]:
# Test de robustesse avec différentes tailles d'échantillon
print("🔬 Test de robustesse - Courbe d'apprentissage...")

from sklearn.model_selection import learning_curve

# Génération de la courbe d'apprentissage
train_sizes, train_scores, val_scores = learning_curve(
    best_model.model, X_test, y_test, 
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=3, scoring='average_precision', n_jobs=-1
)

# Calcul des moyennes et écarts-types
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Visualisation
plt.figure(figsize=(12, 8))
plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Score d\'entraînement')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')

plt.plot(train_sizes, val_mean, 'o-', color='red', label='Score de validation')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')

plt.xlabel('Taille de l\'échantillon d\'entraînement')
plt.ylabel('Score PR-AUC')
plt.title('Courbe d\'Apprentissage - Robustesse du Modèle')
plt.legend()
plt.grid(True)
plt.show()

print("📊 Analyse de la courbe d'apprentissage :")
print(".4f")
print(".4f")
print(".4f")

In [ ]:
# Test de stabilité avec bootstrap
print("🔬 Test de stabilité - Bootstrap...")

n_bootstrap = 100
bootstrap_scores = []

np.random.seed(42)
for i in range(n_bootstrap):
    # Échantillonnage bootstrap
    indices = np.random.choice(len(X_test), size=len(X_test), replace=True)
    X_boot = X_test.iloc[indices]
    y_boot = y_test.iloc[indices]
    
    # Prédiction et calcul du score
    y_pred_boot = best_model.predict(X_boot)
    y_proba_boot = best_model.predict_proba(X_boot)[:, 1]
    
    pr_auc_boot = pr_auc_score(y_boot, y_proba_boot)
    bootstrap_scores.append(pr_auc_boot)

bootstrap_scores = np.array(bootstrap_scores)

print("📊 STATISTIQUES DE STABILITÉ (Bootstrap) :")
print(".4f")
print(".4f")
print(".4f")
print(".4f")
print(".4f")

# Visualisation de la distribution bootstrap
plt.figure(figsize=(10, 6))
plt.hist(bootstrap_scores, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(np.mean(bootstrap_scores), color='red', linestyle='--', linewidth=2, 
            label='.4f')
plt.axvline(np.percentile(bootstrap_scores, 2.5), color='orange', linestyle=':', linewidth=2, 
            label='IC 95%')
plt.axvline(np.percentile(bootstrap_scores, 97.5), color='orange', linestyle=':', linewidth=2)
plt.xlabel('Score PR-AUC')
plt.ylabel('Fréquence')
plt.title('Distribution des Scores Bootstrap')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Analyse des Features Importantes

In [ ]:
# Analyse des features importantes
print("🔍 Analyse des features importantes...")

# Récupération des importances (si disponible)
if hasattr(best_model.model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X_test.columns,
        'importance': best_model.model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("🏆 TOP 15 FEATURES LES PLUS IMPORTANTES :")
    display(feature_importance.head(15))
    
    # Visualisation
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(15)
    sns.barplot(data=top_features, x='importance', y='feature', palette='viridis')
    plt.title('Top 15 Features Importantes')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.show()
    
elif hasattr(best_model.model, 'coef_'):
    # Pour les modèles linéaires
    feature_importance = pd.DataFrame({
        'feature': X_test.columns,
        'coefficient': best_model.model.coef_[0]
    }).sort_values('coefficient', key=abs, ascending=False)
    
    print("🏆 TOP 15 FEATURES PAR COEFFICIENT :")
    display(feature_importance.head(15))
    
else:
    print("⚠️ Méthode d'importance des features non disponible pour ce modèle")

In [ ]:
# Analyse de l'impact des features sur les prédictions
if 'feature_importance' in locals():
    print("📊 Analyse de l'impact des top features...")
    
    # Sélection des 5 features les plus importantes
    top_5_features = feature_importance['feature'].head(5).tolist()
    
    # Analyse de corrélation avec la cible
    correlations = X_test[top_5_features].corrwith(y_test)
    
    plt.figure(figsize=(10, 6))
    correlations.plot(kind='bar', color='coral', alpha=0.7)
    plt.title('Corrélation des Top Features avec la Cible')
    plt.xlabel('Feature')
    plt.ylabel('Corrélation')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("📈 Corrélations avec la variable cible :")
    for feature, corr in correlations.items():
        print("20s")

## 6. Rapport Final et Recommandations

In [ ]:
# Génération du rapport final
print("📋 RAPPORT FINAL D'ÉVALUATION")
print("=" * 60)

print(f"🏆 MODÈLE ÉVALUÉ : {metadata.get('model_name', 'Unknown')}")
print(f"📅 Date d'évaluation : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n📊 MÉTRIQUES PRINCIPALES :")
print("-" * 40)
print(".4f")
print(".4f")
print(".4f")
print(".4f")
print(".4f")

print("\n🔍 ANALYSE DES ERREURS :")
print("-" * 40)
print(f"• Taux d'erreur global : {results_df['error'].mean() * 100:.2f}%")
print(f"• Faux positifs : {len(false_positives)}")
print(f"• Faux négatifs : {len(false_negatives)}")
print(f"• Seuil optimal recommandé : {optimal_threshold:.3f}")

print("\n🛡️ ROBUSTESSE :")
print("-" * 40)
print(".4f")
print(".4f")
print(".4f")

print("\n💡 RECOMMANDATIONS :")
print("-" * 40)

if all_metrics['pr_auc'] > 0.85:
    print("✅ Excellentes performances - Modèle prêt pour la production")
elif all_metrics['pr_auc'] > 0.75:
    print("⚠️ Bonnes performances - Quelques améliorations possibles")
else:
    print("❌ Performances insuffisantes - Révision nécessaire")

if len(false_negatives) > len(false_positives):
    print("⚠️ Attention : Plus de faux négatifs - Risque de fraudes non détectées")
else:
    print("ℹ️ Équilibre acceptable entre faux positifs et faux négatifs")

if bootstrap_scores.std() > 0.05:
    print("⚠️ Variabilité importante - Modèle peut être instable")
else:
    print("✅ Bonne stabilité du modèle")

print("\n🚀 PROCHAINES ÉTAPES :")
print("-" * 40)
print("• Déploiement en production avec monitoring continu")
print("• Collecte de nouvelles données pour réentraînement")
print("• Tests A/B avec différents seuils de décision")
print("• Intégration dans le pipeline de détection temps réel")

print("\n" + "=" * 60)
print("🎯 ÉVALUATION TERMINÉE - MODÈLE PRÊT POUR LA PRODUCTION !")

In [ ]:
# Sauvegarde du rapport d'évaluation
print("💾 Sauvegarde du rapport d'évaluation...")

evaluation_report = {
    'model_info': metadata,
    'evaluation_date': datetime.now().isoformat(),
    'metrics': all_metrics,
    'error_analysis': {
        'total_errors': int(results_df['error'].sum()),
        'error_rate': float(results_df['error'].mean()),
        'false_positives': len(false_positives),
        'false_negatives': len(false_negatives),
        'optimal_threshold': float(optimal_threshold)
    },
    'robustness': {
        'bootstrap_mean': float(bootstrap_scores.mean()),
        'bootstrap_std': float(bootstrap_scores.std()),
        'bootstrap_ci_lower': float(np.percentile(bootstrap_scores, 2.5)),
        'bootstrap_ci_upper': float(np.percentile(bootstrap_scores, 97.5))
    },
    'recommendations': [
        "Modèle prêt pour la production" if all_metrics['pr_auc'] > 0.85 else "Améliorations nécessaires",
        "Monitoring des faux négatifs recommandé" if len(false_negatives) > len(false_positives) else "Équilibre acceptable",
        "Réentraînement périodique conseillé" if bootstrap_scores.std() > 0.05 else "Modèle stable"
    ]
}

# Sauvegarde
report_path = "../reports/evaluation_report.json"
os.makedirs("../reports", exist_ok=True)
with open(report_path, 'w') as f:
    json.dump(evaluation_report, f, indent=2, default=str)

print(f"✅ Rapport sauvegardé : {report_path}")
print("\n🎉 ÉVALUATION COMPLÈTE !")